In [1]:
import pandas as pd
import nltk
import json

## Helper functions
### NLTK Application

In [2]:
# def merge_columns_in_dataframe(df):
#     columns_to_merge = [
#         "pagetitle",
#         "subtitle",
#         "introduction",
#         "summarybox",
#         "content",
#         "accordion",
#     ]
#     new_df = df.loc[:, ["url", "pagetitle"]].copy()
#     new_df["text"] = df.apply(
#         lambda row: " ".join([str(row[col]) for col in columns_to_merge]), axis=1
#     )
#     return new_df


def merge_columns_in_dataframe(df):
    columns_to_merge = [
        "pagetitle",
        "subtitle",
        "introduction",
        "summarybox",
        "content",
        "accordion",
    ]
    new_df = df.loc[:, ["url", "pagetitle"]].copy()

    # Funktion, die die Inhalte zusammenführt und dabei auf Duplikate zwischen `introduction` und `content` achtet
    def merge_row(row):
        # Fügen Sie zuerst die Inhalte hinzu, die sich nicht überschneiden
        non_overlapping_content = " ".join([str(row[col]) for col in columns_to_merge if col not in ['introduction', 'content']])
        
        # Überprüfen, ob der Inhalt von 'introduction' bereits in 'content' enthalten ist
        if pd.notna(row['introduction']) and pd.notna(row['content']):
            if str(row['introduction']) not in str(row['content']):
                combined_content = f"{str(row['introduction'])} {str(row['content'])}"
            else:
                combined_content = str(row['content'])
        else:
            # Wenn einer der Werte NaN ist, verwenden Sie einfach eine direkte Verkettung mit einem Leerzeichen dazwischen
            combined_content = f"{str(row['introduction'])} {str(row['content'])}"

        # Kombinieren Sie den Inhalt aus 'introduction' und 'content' mit den anderen Spalten
        full_text = f"{non_overlapping_content} {combined_content}"
        return full_text

    # Anwenden der `merge_row` Funktion auf jede Zeile
    new_df['text'] = df.apply(merge_row, axis=1)

    return new_df


In [3]:
def nltk_text_preprocessing(text):
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import SnowballStemmer

    tokens = word_tokenize(text.lower())
    filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
    stemmer = SnowballStemmer("german")
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    processed_text = " ".join(stemmed_tokens)
    return processed_text

def get_sentiment(text_string):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()

    scores = analyzer.polarity_scores(text_string)
    return pd.Series(scores)

In [4]:
def add_sentiment_cols(dataframe, analysis_method, text_col="text"):
    if analysis_method == "distilbert":
        sentiment_cols = dataframe[text_col].apply(get_sentiment_distilbert)
    elif analysis_method == "nltk" or analysis_method == "spacy":
        sentiment_cols = dataframe[text_col].apply(get_sentiment)
    else:
        "please provide method (nltk/spacy/distilbert)"
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    # dataframe['judgement'] = dataframe['compound'].apply(lambda x: 'positive' if x > 0.05 else ("negative" if x < -0.05 else 'neutral'))
    return dataframe

In [5]:
def get_analysis_data(json_file, analysis_method):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'pagetitle'])] != "").any(axis=1)] # excludes all columns from the analysis that have "" in the columns except "pagetitle" 
    data = merge_columns_in_dataframe(filtered_df)
    if analysis_method == "nltk":
        data["text"] = data["text"].apply(nltk_text_preprocessing)
    elif analysis_method == "spacy":
        data["text"] = data["text"].apply(spacy_text_preprocessing)
    return add_sentiment_cols(data, analysis_method)

### SpaCy Lemmatization

In [6]:
def spacy_text_preprocessing(text):
    import spacy
    nlp = spacy.load("de_core_news_sm")

    doc = nlp(text.lower())
    filtered_tokens = [token for token in doc if not token.is_stop] # exclusion of stop words

    lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
    processed_text = " ".join(lemmatized_tokens)
    return processed_text

### Huggingface DistillBERT Model Application

In [7]:
def calculate_average_scores(sentiment_results):
    scores = {'negative': [], 'neutral': [], 'positive': []}
    for result in sentiment_results:
        for res in result:
            scores[res['label']].append(res['score'])
    average_scores = {label: sum(score_list) / len(score_list) if score_list else 0 for label, score_list in scores.items()}
    return average_scores


def get_sentiment_distilbert(text_string):
    from transformers import pipeline
    import spacy

    classifier = pipeline('sentiment-analysis',
        model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
        top_k=None,
        truncation=False,
    )

    try:
        nlp = spacy.load('de_core_news_sm')
        doc = nlp(text_string)

        tokens = [token.text for token in doc]
        results = []
        if len(tokens) > 300:
            for sentence in doc.sents:
                result = classifier(sentence.text)
                results.extend(result)
        else:
            result = classifier(text_string)
            results.extend(result)

        average_scores = calculate_average_scores(results)
        return pd.Series(average_scores)
    except Exception as e:
        print(f"Error processing text: {text_string} with error {e}")
        return pd.Series({'negative': 0, 'neutral': 0, 'positive': 0})

In [8]:
def save_as_json(pandas_dataframe, file_name):
    import datetime
    date_info = datetime.datetime.now().strftime("%d-%m-%y")
    pandas_dataframe.to_json(f"{file_name}_{date_info}.json", orient='records')

## Execution of Code

In [9]:
distilbert_results = get_analysis_data("../web-crawler/scrapy_mobiliar/mobiscraper/mobiscraper/spiders/scrape_archive/full_scrape_IV.json", "distilbert")
save_as_json(distilbert_results, "saiv")

/Users/romanroth/opt/anaconda3/envs/WBTH-Project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# save_as_json(pandas_dataframe=distilbert_results, file_name="distilbert_s_a_result")

## Comparing Distilbert / NLTK / SpaCy

In [22]:
get_analysis_data("example.json", "nltk")

,url,pagetitle,text,neg,neu,pos,compound
0,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Kunst & Nachhaltigkeit Vol. 14: 25 Jahre Prix ...,kunst & nachhalt vol . 14 : 25 jahr prix mobil...,0.000,0.987,0.013,0.2263
1,https://www.mobiliar.ch/versicherungen-und-vor...,Gebäudeversicherung – der Rundumschutz für Ihr...,gebaudeversicher – rundumschutz haus finanziel...,0.006,0.982,0.012,0.2869
2,https://www.mobiliar.ch/versicherungen-und-vor...,E-Autos laden: Was Sie alles wissen müssen,e-autos lad : wiss muss elektroautos vormarsch...,0.014,0.951,0.035,0.6767
3,https://www.mobiliar.ch/die-mobiliar/medien/me...,Digitale Identität: Mobiliar beteiligt sich an...,digital identitat : mobiliar beteiligt gemeins...,0.000,0.951,0.049,0.3818
4,https://www.mobiliar.ch/die-mobiliar/karriere/...,System Engineering,syst engineering begleit mobiliar weiterentwic...,0.013,0.970,0.018,0.5994


In [21]:
get_analysis_data("example.json", "spacy")

,url,pagetitle,text,neg,neu,pos,compound
0,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Kunst & Nachhaltigkeit Vol. 14: 25 Jahre Prix ...,Kunst & Nachhaltigkeit vol -- 14 -- 25 prix Mo...,0.000,0.989,0.011,0.2263
1,https://www.mobiliar.ch/versicherungen-und-vor...,Gebäudeversicherung – der Rundumschutz für Ihr...,Gebäudeversicherung -- Rundumschutz Haus finan...,0.005,0.988,0.007,0.0516
2,https://www.mobiliar.ch/versicherungen-und-vor...,E-Autos laden: Was Sie alles wissen müssen,e-autos laden -- wissen Elektroauto Vormarsch ...,0.000,0.990,0.010,0.3612
3,https://www.mobiliar.ch/die-mobiliar/medien/me...,Digitale Identität: Mobiliar beteiligt sich an...,digital Identität -- Mobiliar beteiligen gemei...,0.000,0.958,0.042,0.3818
4,https://www.mobiliar.ch/die-mobiliar/karriere/...,System Engineering,System Engineering begleiten Mobiliar Weiteren...,0.005,0.967,0.028,0.9531


In [20]:
get_analysis_data("example.json", "distilbert")

,url,pagetitle,text,negative,neutral,positive
0,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Kunst & Nachhaltigkeit Vol. 14: 25 Jahre Prix ...,Kunst & Nachhaltigkeit Vol. 14: 25 Jahre Prix ...,0.067596,0.063357,0.869048
1,https://www.mobiliar.ch/versicherungen-und-vor...,Gebäudeversicherung – der Rundumschutz für Ihr...,Gebäudeversicherung – der Rundumschutz für Ihr...,0.430694,0.165596,0.403710
2,https://www.mobiliar.ch/versicherungen-und-vor...,E-Autos laden: Was Sie alles wissen müssen,E-Autos laden: Was Sie alles wissen müssen Ele...,0.346649,0.205312,0.448039
3,https://www.mobiliar.ch/die-mobiliar/medien/me...,Digitale Identität: Mobiliar beteiligt sich an...,Digitale Identität: Mobiliar beteiligt sich an...,0.359737,0.110319,0.529944
4,https://www.mobiliar.ch/die-mobiliar/karriere/...,System Engineering,System Engineering Begleiten Sie die Mobiliar ...,0.256660,0.135608,0.607732


## to display the Functions

In [1]:
text1 = "Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.  Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert."

In [31]:
text2 = "Heute war ein wunderschöner Tag. Ich ging in den Wald spazieren und habe die Tiere beobachtet. Ich freue mich dies bald wieder zu tun."


In [26]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text2.lower())
output_text = " ".join(tokens)
output_text

NameError: name 'input_text' is not defined

In [40]:
from nltk.corpus import stopwords
filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
output_text = " ".join(filtered_tokens)
output_text

'neue « liebe mobiliar » -spot zeigt , schäden hause immer genau passieren , rechnet . mobiliar startet neusten « liebe mobiliar » -spot neue jahr . familie spot hingegen startet ersten blick gewöhnlichen sonntagmorgen . : eltern gemütlich hinschlummern , geräuschen geweckt , definitiv haus gehören . grund dafür tochter , talent eiskunstläuferin entdeckt freude auslebt . hinterlässt spuren . immer kommt – helfen rasch unkompliziert .'

In [41]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("german")
stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
output_text_nltk = " ".join(stemmed_tokens)
output_text_nltk

'neu « lieb mobiliar » -spot zeigt , schad haus imm genau passi , rechnet . mobiliar startet neust « lieb mobiliar » -spot neu jahr . famili spot hingeg startet erst blick gewohn sonntagmorg . : elt gemut hinschlumm , gerausch geweckt , definitiv haus gehor . grund dafur tocht , talent eiskunstlauferin entdeckt freud auslebt . hinterlasst spur . imm kommt – helf rasch unkompliziert .'

In [42]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

scores = analyzer.polarity_scores(output_text_nltk)
pd.Series(scores)

neg         0.0000
neu         0.9460
pos         0.0540
compound    0.4215
dtype: float64

In [43]:
import spacy
nlp = spacy.load("de_core_news_sm")

doc = nlp(text.lower())
filtered_tokens = [token for token in doc if not token.is_stop] # exclusion of stop words

lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
output_text_spacy = " ".join(lemmatized_tokens)
output_text_spacy

'-- liebe mobiliar»-spot zeigen -- schäden Hause genau passieren -- rechnen --   Mobiliar starten neu -- liebe mobiliar»-spot -- Familie Spot hingegen starten Blick gewöhnlich Sonntagmorg -- -- Eltern gemütlich hinschlummern -- geräuschen wecken -- definitiv Haus gehören -- Grund Tochter -- Talent eiskunstläuferin entdecken Freude ausleben -- Hinterlässt Spur -- -- helfen rasch unkompliziert --'

In [44]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

scores = analyzer.polarity_scores(output_text_spacy)
pd.Series(scores)

neg         0.0000
neu         0.9520
pos         0.0480
compound    0.4215
dtype: float64

In [7]:
get_sentiment_distilbert(text1)

negative    0.642108
neutral     0.085077
positive    0.272815
dtype: float64

## Training the Model
Habe ich nicht umgesetzt.

In [5]:
from transformers import AutoTokenizer
from transformers import pipeline
import torch.nn.functional as F
import torch

model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
classifier = pipeline(
    model=model_name,
    tokenizer = AutoTokenizer.from_pretrained(model_name),
    top_k=None
)

X_train = ["Ich bin Roman und 28 Jahre alt.", "Ich liebe dich!"]
res = classifier(X_train)

print(res)
tokenizer = AutoTokenizer.from_pretrained(model_name)
batch = tokenizer(X_train, padding=True, truncation=True, max_length=800, return_tensors="pt")

with torch.no_grad():
    outputs = model_name(**batch),
    print(outputs)
    predictions = F.softmax(output.logits, dim=1),
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)


# list_list_of_dicts_of_tuples = res
# for list_of_dicts_of_tuples in list_list_of_dicts_of_tuples:
#     for dicts in list_of_dicts_of_tuples:
#         mydict[dicts["label"]] = dicts["score"]
# print(pd.Series(mydict))

[[{'label': 'positive', 'score': 0.49148350954055786}, {'label': 'neutral', 'score': 0.25997069478034973}, {'label': 'negative', 'score': 0.2485458105802536}], [{'label': 'positive', 'score': 0.9740146994590759}, {'label': 'neutral', 'score': 0.015805009752511978}, {'label': 'negative', 'score': 0.01018031220883131}]]


TypeError: 'str' object is not callable